# Lesson 2: Tool Calling

## Setup

In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
nest_asyncio.apply()

## 1. Define a Simple Tool

In [3]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int: 
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [19]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "Tell me the output of the mystery function on 2 and 9", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


## 2. Define an Auto-Retrieval Tool

### Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [20]:
from llama_index.core import SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader(input_files=["JM3 3.pdf"]).load_data()

In [21]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [22]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: JM3 3.pdf
file_path: JM3 3.pdf
file_type: application/pdf
file_size: 244011
creation_date: 2024-07-18
last_modified_date: 2024-07-18

Speech and Language Processing. Daniel Jurafsky & James H. Martin. Copyright ©2023. All
rights reserved. Draft of February 3, 2024.
CHAPTER
3N-gram Language Models
“You are uniformly charming!” cried he, with a smile of associating and now
and then I bowed and they perceived a chaise and four to wish for.
Random sentence generated from a Jane Austen trigram model
Predicting is difﬁcult—especially about the future, as the old quip goes. But how
about predicting something that seems much easier, like the next few words someone
is going to say? What word, for example, is likely to follow
Please turn your homework ...
Hopefully, most of you concluded that a very likely word is in, or possibly over,
but probably not refrigerator orthe. In this chapter we formalize this intuition by
introducing models that assign a probability to each 

In [23]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [30]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)

response = query_engine.query(
    "What are N-Grams", 
)

In [31]:
print(str(response))

N-Grams are sequences of n words, where n can be any number. For example, a 2-gram (bigram) is a sequence of two words, like "please turn", while a 3-gram (trigram) is a sequence of three words, such as "please turn your". N-Grams are also used to refer to probabilistic models that estimate the probability of a word given the n-1 previous words, allowing for the calculation of probabilities for entire sequences of words.


In [32]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'JM3 3.pdf', 'file_path': 'JM3 3.pdf', 'file_type': 'application/pdf', 'file_size': 244011, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}


### Define the Auto-Retrieval Tool

In [33]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [39]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "What is N-Grams as described on page 2?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "N-Grams", "page_numbers": ["2"]}
=== Function Output ===
N-grams are sequences of n words, such as bigrams (2-word sequences) and trigrams (3-word sequences). They are used to build language models that estimate the probability of a word given the n-1 previous words. N-grams play a fundamental role in language modeling, helping with tasks like computing probabilities of word sequences, training and testing sets, calculating perplexity, sampling, and interpolation. While n-grams provide a simple formalization for language modeling, more advanced models like neural large language models based on transformer architectures are also used for more complex tasks.


In [40]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "What is N-Grams as described on page 3?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "N-Grams", "page_numbers": ["3"]}
=== Function Output ===
N-Grams are a type of probabilistic language model used in natural language processing. They are based on the idea that the probability of a word in a sequence can be approximated by considering the probability of that word given the previous n-1 words. In the context provided, the bigram model is specifically mentioned, where the probability of a word is approximated by considering only the preceding word. This approach simplifies the computation of probabilities for sequences of words by making the Markov assumption that the probability of a word depends only on the immediately preceding word.


In [45]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "Print the equations on page 3 in latex", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "equations", "page_numbers": ["3"]}
=== Function Output ===
Equations 3.3, 3.4, 3.5, 3.6, and 3.7 are discussed in the provided text.


In [46]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '3', 'file_name': 'JM3 3.pdf', 'file_path': 'JM3 3.pdf', 'file_type': 'application/pdf', 'file_size': 244011, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}


## Let's add some other tools!

In [47]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary"
    ),
)

In [52]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "Get Perplexity on page 8 and summarize it", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "Perplexity", "page_numbers": ["8"]}
=== Function Output ===
Perplexity is a metric used to evaluate language models in natural language processing. It is calculated as the inverse probability of a test set, normalized by the number of words in the set. Essentially, perplexity measures how well a language model predicts a given test set, with lower perplexity values indicating better performance.


In [53]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '8', 'file_name': 'JM3 3.pdf', 'file_path': 'JM3 3.pdf', 'file_type': 'application/pdf', 'file_size': 244011, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}


In [63]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "Summarize", 
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "Summarize"}
=== Function Output ===
N-gram language models are discussed, focusing on predicting word probabilities based on previous words. The models simplify predictions by assuming word probabilities depend on the previous n-1 words. Bigram probabilities are calculated using maximum likelihood estimation. Perplexity is used to evaluate model performance, with lower values indicating better models. Techniques like smoothing and handling unknown words are introduced to address challenges in language models. Vocabulary management, backoff, interpolation, and setting λ values are also discussed in the context of language modeling. The history of n-gram models, advancements in smoothing techniques, and the shift towards neural network models are highlighted. Various approaches to language modeling, including n-grams and neural networks, have been explored over the years to improve word prediction and communica

In [64]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '1', 'file_name': 'JM3 3.pdf', 'file_path': 'JM3 3.pdf', 'file_type': 'application/pdf', 'file_size': 244011, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}
{'page_label': '2', 'file_name': 'JM3 3.pdf', 'file_path': 'JM3 3.pdf', 'file_type': 'application/pdf', 'file_size': 244011, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}
{'page_label': '3', 'file_name': 'JM3 3.pdf', 'file_path': 'JM3 3.pdf', 'file_type': 'application/pdf', 'file_size': 244011, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}
{'page_label': '4', 'file_name': 'JM3 3.pdf', 'file_path': 'JM3 3.pdf', 'file_type': 'application/pdf', 'file_size': 244011, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}
{'page_label': '5', 'file_name': 'JM3 3.pdf', 'file_path': 'JM3 3.pdf', 'file_type': 'application/pdf', 'file_size': 244011, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}
{'page_label': '6', 'file_name': 'JM3 3.

In [65]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "Summarize Perplexity", 
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "Perplexity is a measurement used in natural language processing to evaluate the performance of language models. It indicates how well a probability distribution or language model predicts a sample. A lower perplexity score indicates better performance, as it signifies that the model is more certain about its predictions. Perplexity is calculated as the inverse probability of the test set, normalized by the number of words."}
=== Function Output ===
Perplexity is a metric commonly used in natural language processing to evaluate the performance of language models. It provides insight into how accurately a probability distribution or language model can predict a given sample. A lower perplexity score is preferred as it indicates better performance, suggesting that the model is more confident in its predictions. The calculation of perplexity involves taking the inverse probability of the test set and normalizing 

In [66]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '1', 'file_name': 'JM3 3.pdf', 'file_path': 'JM3 3.pdf', 'file_type': 'application/pdf', 'file_size': 244011, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}
{'page_label': '2', 'file_name': 'JM3 3.pdf', 'file_path': 'JM3 3.pdf', 'file_type': 'application/pdf', 'file_size': 244011, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}
{'page_label': '3', 'file_name': 'JM3 3.pdf', 'file_path': 'JM3 3.pdf', 'file_type': 'application/pdf', 'file_size': 244011, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}
{'page_label': '4', 'file_name': 'JM3 3.pdf', 'file_path': 'JM3 3.pdf', 'file_type': 'application/pdf', 'file_size': 244011, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}
{'page_label': '5', 'file_name': 'JM3 3.pdf', 'file_path': 'JM3 3.pdf', 'file_type': 'application/pdf', 'file_size': 244011, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}
{'page_label': '6', 'file_name': 'JM3 3.